In [87]:
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

CSV를 가져오고 병합하는 부분까지는 생략하겠다.

In [88]:
credits = pd.read_csv("credits.csv")
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [89]:
md = pd.read_csv('movies_metadata.csv')
link_small = pd.read_csv('links_small.csv')
link_small = link_small[link_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
movies_df = md[md['id'].isin(link_small)]

C:\Users\arjec\AppData\Local\Temp\ipykernel_8904\821452009.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('movies_metadata.csv')


In [90]:
import json

with open('keywords.json', encoding='utf-8') as f:
  js = json.loads(f.read())
keywords_df = pd.DataFrame(js)

In [91]:
movies_df = movies[['title', 'id', 'genres']]
credits_df = credits[['cast', 'crew', 'id']]


In [92]:
movies_df.id = pd.to_numeric(movies_df.id, errors='coerce')
credits_df.id = pd.to_numeric(credits_df.id, errors='coerce')
keywords_df.id = pd.to_numeric(keywords_df.id, errors='coerce')

C:\Users\arjec\AppData\Local\Temp\ipykernel_8904\4167628614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.id = pd.to_numeric(movies_df.id, errors='coerce')


In [93]:
movies_df = pd.merge(movies_df, credits_df, on='id', how='inner')
movies_df.head() 

,title,id,genres,cast,crew
0,Toy Story,862.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,Jumanji,8844.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,Grumpier Old Men,15602.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,Waiting to Exhale,31357.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,Father of the Bride Part II,11862.0,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [94]:
movies_df = pd.merge(movies_df, keywords_df, on='id', how='inner')
movies_df.head()

,title,id,genres,cast,crew,keywords
0,Toy Story,862.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,8844.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,15602.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,31357.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,11862.0,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


배우와 감독 정제하기

In [95]:
from ast import literal_eval

movies_df['cast'] = movies_df['cast'].apply(literal_eval)
movies_df['crew'] = movies_df['crew'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['cast_size'] = movies_df['cast'].apply((lambda x :len(x)))
movies_df['crew_size'] = movies_df['crew'].apply((lambda x :len(x)))

MemoryError: 

In [54]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [55]:
movies_df['director'] = movies_df['crew'].apply(get_director)

In [56]:
movies_df['cast'] = movies_df['cast'].apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

In [57]:
movies_df['cast'] = movies_df['cast'].apply(lambda x: x[:3] if len(x) >3 else x)

In [58]:
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [59]:
movies_df['cast'] = movies_df['cast'].apply(lambda x : [str.lower(i.replace(" ", "")) for i in x])
movies_df['director'] = movies_df['director'].astype(str).apply(lambda x: str.lower(x.replace(" ", "")))
movies_df['director'] = movies_df['director'].apply(lambda x: [x, x, x])

In [60]:
s = movies_df.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keywords'

C:\Users\arjec\AppData\Local\Temp\ipykernel_8904\612500976.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movies_df.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)


In [61]:
s = s.value_counts()

In [62]:
s = s[s>1]

In [63]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [64]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [73]:
movies_df['keywords'] = movies_df['keywords'].apply(filter_keywords)
movies_df.head()

,title,id,genres,cast,crew,keywords,cast_size,crew_size,director
0,Toy Story,862.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[toy, boy, friendship, friend]",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,Jumanji,8844.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",[],26,16,"[joejohnston, joejohnston, joejohnston]"
2,Grumpier Old Men,15602.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...",[fish],7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,Waiting to Exhale,31357.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...",[],10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,Father of the Bride Part II,11862.0,"[{'id': 35, 'name': 'Comedy'}]","[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[daughter, gynecologist]",12,7,"[charlesshyer, charlesshyer, charlesshyer]"


In [76]:
movies_df['genres'] = movies_df['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

In [77]:
movies_df['keywords'] = movies_df['keywords'].apply(lambda x :[stemmer.stem(i) for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [79]:
movies_df['soup'] = movies_df['keywords']+movies_df['cast']+movies_df['director']+movies_df['genres']
movies_df['soup'] = movies_df['soup'].apply(lambda x: ' '.join(x))

In [83]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(movies_df['soup'])

In [86]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

MemoryError: Unable to allocate 5.80 GiB for an array with shape (778921872,) and data type float64

In [ ]:
movies_df = movies_df.reset_index()
titles = movies_df['title']
indces = pd.Series(movies_df.index, index=movies_df['title'])

In [ ]:
def getrecommandations(title):
    index = indces[title]
    movies_df_scores = list(enumerate(cosine_sim[index]))
    movies_df_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    movies_df_scores = sim_scores[1:31]
    movies_df_indices = [i[0] for i in sim_scores] 
    return titles.iloc[movies_df_indices] 

In [ ]:
getrecommandations('The Dark Knight')